In [2]:
import numpy
import os
import pandas
import matplotlib.pyplot as plt
import sklearn.model_selection as model_selection

from datetime import *

from sklearn.preprocessing import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *

In [3]:
df = pandas.read_csv('animal_chess.csv')

In [4]:
count = len(df)

In [5]:
sample = df.iloc[32]
sample

board     ----r-E-Tl-----WC--d---------p------L--R--P---...
side                                                     -1
piece                                                     L
atk                                                       0
move                                                   F1E1
river                                                     0
trap                                                      1
den                                                       0
score                                                   -90
winner                                                    0
Name: 32, dtype: object

In [6]:
# Encode the piece
def encode_piece(piece_char):
    piece_mapping = {'-': 0, 'r': 1, 'c': 2, 'd': 3, 'w': 4, 'p': 5, 't': 6, 'l': 7, 'e': 8, 'R': -1, 'C': -2, 'D': -3, 'W': -4, 'P': -5, 'T': -6, 'L': -7, 'E': -8}
    return piece_mapping.get(piece_char, 0)

encode_piece(sample['piece'])

-7

In [7]:
# Encode the board
def encode_board(board_str):
    board_matrix = numpy.zeros((9, 7))
    for i, piece in enumerate(board_str[::-1]):
        row, col = divmod(i, 9)
        board_matrix[col][row] = encode_piece(piece)
    return numpy.flip(numpy.flip(board_matrix, 0), 1)

encode_board(sample['board'])

array([[ 0.,  7.,  0.,  0., -7.,  0.,  0.],
       [ 0.,  0.,  3.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  5.,  0.,  0.,  4.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-8., -4.,  0.,  0., -5.,  0.,  0.],
       [ 0., -2.,  0.,  0.,  0., -3.,  0.],
       [-6.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [9]:
# Encode the move
def encode_move(move_str):
    col_from = ord(move_str[0]) - ord('A')
    row_from = int(move_str[1]) - 1
    col_to = ord(move_str[2]) - ord('A')
    row_to = int(move_str[3]) - 1
    return [row_from, col_from, row_to, col_to]

encode_move(sample['move'])

[0, 5, 0, 4]

In [8]:
df['board_encoded'] = df['board'].apply(encode_board)
board_matrix_flattened = numpy.array(df['board_encoded'].tolist()).reshape(count, -1)
board_matrix_flattened

array([[ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [10]:
df['move_encoded'] = df['move'].apply(encode_move)
move_vector_flattened = numpy.array(df['move_encoded'].tolist()).reshape(count, -1)
move_vector_flattened

array([[6, 6, 5, 6],
       [2, 0, 3, 0],
       [5, 6, 4, 6],
       ...,
       [2, 3, 1, 3],
       [3, 3, 2, 3],
       [1, 3, 0, 3]])

In [11]:
ohe = OneHotEncoder()
categorical_features = ohe.fit_transform(df[['side', 'piece', 'river', 'trap', 'den']]).toarray()
categorical_features

array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [12]:
scaler = MinMaxScaler()
numeric_features = scaler.fit_transform(df[['atk']])
numeric_features

array([[0.125],
       [0.125],
       [0.125],
       ...,
       [0.   ],
       [0.625],
       [0.125]])

In [13]:
X = numpy.concatenate([board_matrix_flattened, move_vector_flattened, categorical_features, numeric_features], axis=1)
y = df['score'].values

In [14]:
# Split the data
def build_model(input_shape):
    inputs = Input(shape=(input_shape,))
    
    x = Dense(512, kernel_regularizer=l2(0.005))(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    
    x = Dense(1024, kernel_regularizer=l2(0.005))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    
    output = Dense(1, activation='linear', name='output')(x)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss=MeanSquaredError(), metrics=['mae'])

    return model

In [15]:
model = build_model(X.shape[1])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 94)]              0         
                                                                 
 dense (Dense)               (None, 512)               48640     
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1024)              525312    
                                                            

In [16]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
tensorboard_callback = TensorBoard(log_dir=os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S")), histogram_freq=1)

In [17]:
X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.1, random_state=42)

In [18]:
history = model.fit(X_train, y_train, epochs=50, batch_size=512, validation_data=(X_val, y_val), callbacks=[reduce_lr, early_stopping, tensorboard_callback])

Epoch 1/50


72590/72590 [==============================] - 4198s 58ms/step - loss: 319.0370 - mae: 8.7531 - val_loss: 21.7008 - val_mae: 2.7512 - lr: 1.0000e-04
Epoch 2/50
72590/72590 [==============================] - 2379s 33ms/step - loss: 101.8053 - mae: 6.2525 - val_loss: 47.4814 - val_mae: 4.2650 - lr: 1.0000e-04
Epoch 3/50
72590/72590 [==============================] - 2430s 33ms/step - loss: 83.7676 - mae: 5.6266 - val_loss: 115.4867 - val_mae: 7.8190 - lr: 1.0000e-04
Epoch 4/50
72590/72590 [==============================] - 2399s 33ms/step - loss: 74.2059 - mae: 5.2596 - val_loss: 84.2263 - val_mae: 6.6165 - lr: 1.0000e-04
Epoch 5/50
72590/72590 [==============================] - 2396s 33ms/step - loss: 67.8318 - mae: 4.9953 - val_loss: 101.8366 - val_mae: 7.1058 - lr: 1.0000e-04
Epoch 6/50
72590/72590 [==============================] - 2393s 33ms/step - loss: 61.9204 - mae: 4.7252 - val_loss: 61.0445 - val_mae: 5.4469 - lr: 1.0000e-04
Epoch 7/50
72590/72590 [================

In [ ]:
model.save('animal_chess_model.h5')

In [ ]:
epochs = range(1, len(history.history['loss']) + 1)
loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mae']
val_mae = history.history['val_mae']

In [ ]:
# Plotting training and validation loss
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting training and validation MAE
plt.subplot(1, 2, 2)
plt.plot(epochs, mae, 'b-', label='Training MAE')
plt.plot(epochs, val_mae, 'r-', label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()